
**Hypothesis**: Mean housing prices are less effected by recessions in university towns. A t-test is run to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. 


In [1]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

In [2]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
inv_states = {v:k for k , v in states.items()}

In [13]:
def get_list_of_university_towns():

    ut = []
    uni_town = open("university_towns.txt","r+")
    for lines in uni_town: 
        ut.append(lines.strip("\n").strip("\t"))
    ut = list(ut)    

    state = []
    town = []

    for i in range(0,len(ut)):
        if "[ed" in ut[i]:
            st = ut[i] 
    
        else: 
            tw = ut[i]
            town.append(tw)
            state.append(st)

    df = pd.DataFrame() 
    df['State'] = state
    df['RegionName'] = town
    df['State'] = df['State'].apply(lambda x: x.split('[')[0]).map(str.strip)
    df['RegionName'] = df['RegionName'].apply(lambda x: x.split('(')[0]).map(str.strip)

 
    return df
    
    
get_list_of_university_towns()


In [4]:
def get_recession_start():
    gdp_ot = pd.read_excel('gdplev.xls',skiprows=7).drop(["Unnamed: 0","Unnamed: 1","Unnamed: 2","Unnamed: 3",
     "Unnamed: 5", "Unnamed: 7"],axis=1)
    gdp_ot = pd.DataFrame(gdp_ot).rename(columns={'Unnamed: 4': 'Year.Q', "Unnamed: 6" : 'GDP'})
    
    gdp_ot = (gdp_ot[list(gdp_ot['Year.Q']).index("2000q1"):len(gdp_ot)]).reset_index(drop=True)
    #this will cut out all the rows except iss to the end
    gdp = gdp_ot['GDP']
    yearq = gdp_ot['Year.Q']
    result = []
    for i in range(0,len(gdp)-2): 
        if (gdp[i+1] < gdp[i]) & (gdp[i+2] < gdp[i+1]):
            result.append(yearq[i+1])
    
    recession_start=result[0]
    
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    return recession_start
    #return gdp_ot[25:len(gdp_ot)]
    
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    gdp_ot = pd.read_excel('gdplev.xls',skiprows=7).drop(["Unnamed: 0","Unnamed: 1","Unnamed: 2","Unnamed: 3",
     "Unnamed: 5", "Unnamed: 7"],axis=1)
    gdp_ot = pd.DataFrame(gdp_ot).rename(columns={'Unnamed: 4': 'Year.Q', "Unnamed: 6" : 'GDP'}) 
    gdp_ot = (gdp_ot[list(gdp_ot['Year.Q']).index(get_recession_start()):len(gdp_ot)]).reset_index(drop=True)
    gdp = gdp_ot['GDP']
    yearq = gdp_ot['Year.Q']
    result = []
    for i in range(0,len(gdp)-2): 
        if (gdp[i+1] > gdp[i]) & (gdp[i+2] > gdp[i+1]):
            result.append(yearq[i+2])
            
    recession_end = result[0]
    
    
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return recession_end

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    gdp_ot = pd.read_excel('gdplev.xls',skiprows=7).drop(["Unnamed: 0","Unnamed: 1","Unnamed: 2","Unnamed: 3",
     "Unnamed: 5", "Unnamed: 7"],axis=1)
    gdp_ot = pd.DataFrame(gdp_ot).rename(columns={'Unnamed: 4': 'Year.Q', "Unnamed: 6" : 'GDP'}) 
    gdp_ot = (gdp_ot[list(gdp_ot['Year.Q']).index(get_recession_start()):list(gdp_ot['Year.Q']).
                     index(get_recession_end())]).reset_index(drop=True)
    yearQ = gdp_ot['Year.Q']
    recession_bottom = yearQ[list(gdp_ot['GDP']).index(np.min(gdp_ot['GDP']))]
    
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    return recession_bottom

get_recession_bottom()


'2009q2'

In [14]:
def convert_housing_data_to_quarters():
    hp2 = pd.read_csv("City_Zhvi_AllHomes.csv")
    df = pd.DataFrame(hp2)
    df['State'] = df['State'].map(states)
    df = df.set_index(['State','RegionName'])
    df = df.ix[:,'2000-01':]
    df.columns = pd.to_datetime(df.columns) 
    mdf = df.resample('Q',axis=1).mean() #resample Q: quarterly 
    col = []
    for cols in mdf.columns:
        cols = '{:}q{:}'.format(cols.year,cols.quarter)
        col.append(cols)
    mdf.columns = col   
     
    
    return mdf

convert_housing_data_to_quarters() 

In [16]:
def run_ttest():
    
    hdf = convert_housing_data_to_quarters()
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    ul = get_list_of_university_towns()
    rec_start = get_recession_start() 
    hdf = convert_housing_data_to_quarters().rename(columns=lambda x: str(x).lower())
    
    qrt = pd.Series(hdf.columns)
    for r in range(0,len(qrt)): 
        if qrt[r] == rec_start: 
            i = r 
            
    qrt_before_recession = qrt[i-1]
    hdf['PriceRatio'] = hdf[qrt_before_recession].div(hdf[rec_bottom])
    ul['State'] = ul['State'].map(str.strip)
    ul['RegionName'] = ul['RegionName'].map(str.strip)                                 
    arrays = [ul['State'],ul['RegionName']]
    tuples = list(zip(*arrays))
    index = pd.MultiIndex.from_tuples(tuples, names=['State', 'RegionName'])
   
    
    uni_towns = hdf.loc[hdf.index.isin(tuples)]


    non_uni_towns = hdf.loc[~hdf.index.isin(tuples)] 
    non_uni_towns['Price ratio'] = non_uni_towns[qrt_before_recession].div(non_uni_towns[rec_bottom])
    uni_towns['Price ratio'] = uni_towns[qrt_before_recession].div(uni_towns[rec_bottom])
    ttest = ttest_ind(non_uni_towns['Price ratio'],uni_towns['Price ratio'],nan_policy='omit')
    pvalue = ttest[1]
    different = pvalue < 0.01 
    compare = uni_towns['Price ratio'].mean() > non_uni_towns['Price ratio'].mean()
    if (compare == 'True'):
        better = "non-university town"
    else: 
        better = "university town"
    
    result = [different,pvalue,better]

   
    
    return result

run_ttest() 